In [109]:
import os

import tensorflow as tf

from os import makedirs
from numpy import zeros
from numpy import ones
from numpy import expand_dims
from numpy import hstack
from numpy.random import randn
from numpy.random import randint
# from tensorflow.keras.datasets.mnist import load_data
from VisionEngine.datasets import guppies
from tensorflow.keras import backend
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.constraints import Constraint
from matplotlib import pyplot
from tensorflow.keras.optimizers import RMSprop
import tensorflow.keras.backend as K
from tensorflow.keras.layers import concatenate

In [110]:
tf.__version__

'2.0.0'

In [111]:
tf.executing_eagerly()


True

In [119]:
import numpy as np
import math
import matplotlib.pyplot as plt
import os

def generator(inputs,
              image_size,
              activation='sigmoid',
              labels=None,
              codes=None):
    """Build a Generator Model
    Stack of BN-ReLU-Conv2DTranpose to generate fake images.
    Output activation is sigmoid instead of tanh in [1].
    Sigmoid converges easily.
    # Arguments
        inputs (Layer): Input layer of the generator (the z-vector)
        image_size (int): Target size of one side (assuming square image)
        activation (string): Name of output activation layer
        labels (tensor): Input labels
        codes (list): 2-dim disentangled codes for InfoGAN
    # Returns
        Model: Generator Model
    """
    image_resize = image_size // 4
    # network parameters
    kernel_size = 4
    layer_filters = [128, 64, 32, 3]

    if labels is not None:
        if codes is None:
            # ACGAN labels
            # concatenate z noise vector and one-hot labels
            inputs = [inputs, labels]
        else:
            # infoGAN codes
            # concatenate z noise vector, one-hot labels and codes 1 & 2
            inputs = [inputs, labels] + codes
        x = concatenate(inputs, axis=1)
    elif codes is not None:
        # generator 0 of StackedGAN
        inputs = [inputs, codes]
        x = concatenate(inputs, axis=1)
    else:
        # default input is just 100-dim noise (z-code)
        x = inputs

    x = Dense(image_resize * image_resize * layer_filters[0])(x)
    x = BatchNormalization()(x)
    x = Reshape((image_resize, image_resize, layer_filters[0]))(x)

    for filters in layer_filters:
        # first two convolution layers use strides = 2
        # the last two use strides = 1
        if filters > layer_filters[-2]:
            strides = 2
        else:
            strides = 1
        x = UpSampling2D(size=2)(x)
        x = Conv2D(filters=filters,
                            kernel_size=kernel_size,
                            strides=strides,
                            padding='same')(x)
        x = BatchNormalization(momentum=0.8)(x)
        x = Activation(activation)(x)

#     if activation is not None:
#         x = Activation(activation)(x)

    # generator output is the synthesized image x
    return Model(inputs, x, name='generator')


def discriminator(inputs,
                  activation='sigmoid',
                  num_labels=8,
                  num_codes=None):
    """Build a Discriminator Model
    Stack of LeakyReLU-Conv2D to discriminate real from fake
    The network does not converge with BN so it is not used here
    unlike in [1]
    # Arguments
        inputs (Layer): Input layer of the discriminator (the image)
        activation (string): Name of output activation layer
        num_labels (int): Dimension of one-hot labels for ACGAN & InfoGAN
        num_codes (int): num_codes-dim Q network as output 
                    if StackedGAN or 2 Q networks if InfoGAN
                    
    # Returns
        Model: Discriminator Model
    """
    kernel_size = 4
    layer_filters = [32, 64, 128, 256]

    x = inputs
    for filters in layer_filters:
        # first 3 convolution layers use strides = 2
        # last one uses strides = 1
        if filters == layer_filters[-1]:
            strides = 1
        else:
            strides = 2
        
        x = Conv2D(filters=filters,
                   kernel_size=kernel_size,
                   strides=strides,
                   padding='same')(x)
        x = LeakyReLU(alpha=0.2)(x)
        x = Dropout(0.25)(x)

    x = Flatten()(x)
    # default output is probability that the image is real
    output = Dense(1)(x)
    if activation is not None:
        print(activation)
        output = Activation(activation)(output)
    
    outputs = []
    outputs.append(output)
    if num_labels:
        # ACGAN and InfoGAN have 2nd output
        # 2nd output is 10-dim one-hot vector of label
        layer = Dense(layer_filters[-2])(x)
        labels = Dense(num_labels)(layer)
        labels = Activation('softmax', name='label')(labels)
        outputs.append(labels)
        for code_i in range(num_codes):
        # InfoGAN have 3rd and 4th outputs
        # 3rd output is 1-dim continous Q of 1st c given x
            code = Dense(1)(layer)
            code = Activation('sigmoid', name='code{}'.format(code_i))(code)
            outputs.append(code)
    
    elif num_codes is not None:
        # StackedGAN Q0 output
        # z0_recon is reconstruction of z0 normal distribution
        z0_recon =  Dense(num_codes)(x)
        z0_recon = Activation('tanh', name='z0')(z0_recon)
        outputs.append(z0_recon)
        
    return Model(inputs, outputs, name='discriminator')



# def train(models, x_train, params):
#     """Train the Discriminator and Adversarial Networks
#     Alternately train Discriminator and Adversarial networks by batch.
#     Discriminator is trained first with properly real and fake images.
#     Adversarial is trained next with fake images pretending to be real
#     Generate sample images per save_interval.
#     # Arguments
#         models (list): Generator, Discriminator, Adversarial models
#         x_train (tensor): Train images
#         params (list) : Networks parameters
#     """
#     # the GAN models
#     generator, discriminator, adversarial = models
#     # network parameters
#     batch_size, latent_size, train_steps, model_name = params
#     # the generator image is saved every 500 steps
#     save_interval = 500
#     # noise vector to see how the generator output evolves during training
#     noise_input = np.random.uniform(-1.0, 1.0, size=[16, latent_size])
#     # number of elements in train dataset
#     train_size = x_train.shape[0]
#     for i in range(train_steps):
#         # train the discriminator for 1 batch
#         # 1 batch of real (label=1.0) and fake images (label=0.0)
#         # randomly pick real images from dataset
#         rand_indexes = np.random.randint(0, train_size, size=batch_size)
#         real_images = x_train[rand_indexes]
#         # generate fake images from noise using generator 
#         # generate noise using uniform distribution
#         noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
#         # generate fake images
#         fake_images = generator.predict(noise)
#         # real + fake images = 1 batch of train data
#         x = np.concatenate((real_images, fake_images))
#         # label real and fake images
#         # real images label is 1.0
#         y = np.ones([2 * batch_size, 1])
#         # fake images label is 0.0
#         y[batch_size:, :] = 0.0
#         # train discriminator network, log the loss and accuracy
#         loss, acc = discriminator.train_on_batch(x, y)
#         log = "%d: [discriminator loss: %f, acc: %f]" % (i, loss, acc)

#         # train the adversarial network for 1 batch
#         # 1 batch of fake images with label=1.0
#         # since the discriminator weights are frozen in adversarial network
#         # only the generator is trained
#         # generate noise using uniform distribution
#         noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
#         # label fake images as real or 1.0
#         y = np.ones([batch_size, 1])
#         # train the adversarial network 
#         # note that unlike in discriminator training, 
#         # we do not save the fake images in a variable
#         # the fake images go to the discriminator input of the adversarial
#         # for classification
#         # log the loss and accuracy
#         loss, acc = adversarial.train_on_batch(noise, y)
#         log = "%s [adversarial loss: %f, acc: %f]" % (log, loss, acc)
#         print(log)
#         if (i + 1) % save_interval == 0:
#             if (i + 1) == train_steps:
#                 show = True
#             else:
#                 show = False

#             # plot generator images on a periodic basis
#             plot_images(generator,
#                         noise_input=noise_input,
#                         show=show,
#                         step=(i + 1),
#                         model_name=model_name)
#             generator.save(model_name + ".h5")
   
#     # save the model after training the generator
#     # the trained generator can be reloaded for future MNIST digit generation
#     generator.save(model_name + ".h5")


def plot_images(generator,
                noise_input,
                noise_label=None,
                noise_codes=None,
                show=False,
                step=0,
                model_name="gan"):
    """Generate fake images and plot them
    For visualization purposes, generate fake images
    then plot them in a square grid
    # Arguments
        generator (Model): The Generator Model for fake images generation
        noise_input (ndarray): Array of z-vectors
        show (bool): Whether to show plot or not
        step (int): Appended to filename of the save images
        model_name (string): Model name
    """
    os.makedirs(model_name, exist_ok=True)
    filename = os.path.join(model_name, "%05d.png" % step)
    rows = int(math.sqrt(noise_input.shape[0]))
    if noise_label is not None:
        noise_input = [noise_input, noise_label]
        if noise_codes is not None:
            noise_input += noise_codes

    images = generator.predict(noise_input)
    plt.figure(figsize=(10, 10))
    num_images = images.shape[0]
    image_size = images.shape[1]
    for i in range(num_images):
        plt.subplot(rows, rows, i + 1)
        image = np.reshape(images[i], [image_size, image_size, 3])
        plt.imshow(image)
        plt.axis('off')
    plt.savefig(filename)
    if show:
        plt.show()
    else:
        plt.close('all')

def test_generator(generator):
    noise_input = np.random.uniform(-0.5, 0.5, size=[16, 100])
    plot_images(generator,
                noise_input=noise_input,
                show=True,
                model_name="test_outputs")

In [120]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import numpy as np
import argparse

import sys
sys.path.append("..")

def train(models, x_train, params):
    """Train the Discriminator and Adversarial Networks
    Alternately train Discriminator and Adversarial networks by batch.
    Discriminator is trained first with properly real and fake images.
    Adversarial is trained next with fake images pretending to be real
    Generate sample images per save_interval.
    # Arguments
        models (list): Generator, Discriminator, Adversarial models
        x_train (tensor): Train images
        params (list) : Networks parameters
    """
    # the GAN models
    generator, discriminator, adversarial = models
    # network parameters
    batch_size, latent_size, train_steps, num_labels, model_name = params
    # the generator image is saved every 500 steps
    save_interval = 500
    # noise vector to see how the generator output evolves during training
    noise_input = np.random.uniform(-1.0, 1.0, size=[16, latent_size])
    # number of elements in train dataset
    train_size = len(x_train)
    for i in range(train_steps):
        # train the discriminator for 1 batch
        # 1 batch of real (label=1.0) and fake images (label=0.0)
        # randomly pick real images from dataset
        rand_indexes = np.random.randint(0, train_size, size=batch_size)
        real_images = x_train[rand_indexes]
        # generate fake images from noise using generator 
        # generate noise using uniform distribution
        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
        # generate fake images
        fake_images = generator.predict(noise)
        # real + fake images = 1 batch of train data
        x = np.concatenate((real_images, fake_images))
        # label real and fake images
        # real images label is 1.0
        y = np.ones([2 * batch_size, 1])
        # fake images label is 0.0
        y[batch_size:, :] = 0.0
        # train discriminator network, log the loss and accuracy
        loss, acc = discriminator.train_on_batch(x, y)
        log = "%d: [discriminator loss: %f, acc: %f]" % (i, loss, acc)

        # train the adversarial network for 1 batch
        # 1 batch of fake images with label=1.0
        # since the discriminator weights are frozen in adversarial network
        # only the generator is trained
        # generate noise using uniform distribution
        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
        # label fake images as real or 1.0
        y = np.ones([batch_size, 1])
        # train the adversarial network 
        # note that unlike in discriminator training, 
        # we do not save the fake images in a variable
        # the fake images go to the discriminator input of the adversarial
        # for classification
        # log the loss and accuracy
        loss, acc = adversarial.train_on_batch(noise, y)
        log = "%s [adversarial loss: %f, acc: %f]" % (log, loss, acc)
        print(log)
        if (i + 1) % save_interval == 0:
            if (i + 1) == train_steps:
                show = True
            else:
                show = False

            # plot generator images on a periodic basis
            plot_images(generator,
                        noise_input=noise_input,
                        show=show,
                        step=(i + 1),
                        model_name=model_name)
            generator.save(model_name + ".h5")
   
    # save the model after training the generator
    # the trained generator can be reloaded for future MNIST digit generation
    generator.save(model_name + ".h5")


def mi_loss(c, q_of_c_given_x):
    """ Mutual information, Equation 5 in [2] , assuming H(c) is constant"""
    # mi_loss = -c * log(Q(c|x))
    return K.mean(-K.sum(K.log(q_of_c_given_x + K.epsilon()) * c, axis=1))


def test_generator(generator, params, latent_size=100):
    label, code1, code2, p1, p2 = params
    noise_input = np.random.uniform(-1., 1., size=[16, latent_size])
    step = 0
    if label is None:
        num_labels = 13
        noise_label = np.eye(num_labels)[np.random.choice(num_labels, 16)]
    else:
        noise_label = np.zeros((16, 13))
        noise_label[:,label] = 1
        step = label

    if code1 is None:
        noise_code1 = np.ones((16, 1)) * -1
    else:
        if p1:
            a = np.linspace(-1, 1, 16)
            a = np.reshape(a, [16, 1])
            noise_code1 = np.ones((16, 1)) * a
        else:
            noise_code1 = np.ones((16, 1)) * code1
        print(noise_code1)

    if code2 is None:
        noise_code2 = np.ones((16, 1)) * -1
    else:
        if p2:
            a = np.linspace(-1, 1, 16)
            a = np.reshape(a, [16, 1])
            noise_code2 = np.ones((16, 1)) * a
        else:
            noise_code2 = np.ones((16, 1)) * code2
        print(noise_code2)

    plot_images(generator,
                    noise_input=noise_input,
                    noise_label=noise_label,
                    noise_codes=[noise_code1, noise_code2],
                    show=False,
                    step=step,
                    model_name="test_outputs_{}".format(label))

In [121]:
pwd

'/home/etheredge/Workspace/VisionEngine/notebooks'

In [122]:
# load dataset
with np.load('../VisionEngine/data_loaders/datasets/guppies.npz') as f:
    x_train = f['x_train']
    y_train = f['y_train']

In [123]:
x_train = x_train
y_train = np.array([y.split('_')[0] for y in y_train])

In [124]:
tf.debugging.set_log_device_placement(True)
with tf.device('/device:GPU:1'):
    model_name = "infogan_guppies"

    # reshape data for CNN as (28, 28, 1) and normalize
    image_size = x_train.shape[1]
    x_train = np.reshape(x_train, [-1, image_size, image_size, 3])
    x_train = x_train.astype('float32')


    ####
    #plotting sanity check

    os.makedirs(model_name, exist_ok=True)
    filename = os.path.join(model_name, "example_real_inputs.png")
    rows = int(math.sqrt(16))
    images = x_train[0:16]
    plt.figure(figsize=(10, 10))
    num_images = 16
    image_size = 256
    for i in range(num_images):
        plt.subplot(rows, rows, i + 1)
        image = np.reshape(images[i], [image_size, image_size, 3])
        plt.imshow(image)
        plt.axis('off')
    plt.savefig(filename)
    plt.close('all')
    ####

    # train labels
    num_labels = len(np.unique(y_train))
    num_codes = 10
    print(num_labels)
    _, y_train = np.unique(y_train,return_inverse=True)
    y_train = to_categorical(y_train)

    # network parameters
    batch_size = 32
    train_steps = 100000
    lr = 2e-4
    decay = 6e-8
    input_shape = (image_size, image_size, 3)
    label_shape = (num_labels, )
    code_shape = (1, )
    latent_size = 100

    # build discriminator model
    inputs = Input(shape=input_shape, name='discriminator_input')
    discriminator = discriminator(inputs,
                                      num_labels=num_labels,
                                      num_codes=num_codes)
    print(discriminator.outputs,discriminator.inputs)
    # [1] uses Adam, but discriminator converges easily with RMSprop
    optimizer = RMSprop(lr=lr, decay=decay)
    # loss functions: 1) probability image is real (binary crossentropy)
    # 2) categorical cross entropy image label,
    # 3) and 4) mutual information loss
    loss = ['binary_crossentropy', 'categorical_crossentropy']
    for i in range(num_codes):
        loss.append(mi_loss)
    # lamda or mi_loss weight is 0.5
    loss_weights = [1.0, 1.0] + [1.0 for i in range(num_codes)]
    discriminator.compile(loss=loss,
                          loss_weights=loss_weights,
                          optimizer=optimizer,
                          metrics=['accuracy'])
    discriminator.summary()

    # build generator model
    input_shape = (latent_size, )
    inputs = Input(shape=input_shape, name='z_input')
    labels = Input(shape=label_shape, name='labels')
    codes = [Input(shape=code_shape, name="code{}".format(i)) for i in range(num_codes)]
    # code1 = Input(shape=code_shape, name="code1")
    # code2 = Input(shape=code_shape, name="code2")
    # call generator with inputs, 
    # labels and codes as total inputs to generator
    generator = generator(inputs,
                              image_size,
                              labels=labels,
                              codes=codes)
    generator.summary()

    # build adversarial model = generator + discriminator
    optimizer = RMSprop(lr=lr*0.5, decay=decay*0.5)
    discriminator.trainable = False
    # total inputs = noise code, labels, and codes
    inputs = [inputs, labels] + codes
    adversarial = Model(inputs,
                        discriminator(generator(inputs)),
                        name=model_name)
    # same loss as discriminator
    adversarial.compile(loss=loss,
                        loss_weights=loss_weights,
                        optimizer=optimizer,
                        metrics=['accuracy'])
    adversarial.summary()

    # train discriminator and adversarial networks
    models = (generator, discriminator, adversarial)
    data = (x_train, y_train)
    params = (batch_size, latent_size, train_steps, num_labels, model_name)
    train(models, data, params)

19
sigmoid
[<tf.Tensor 'activation_60/Identity:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'label_12/Identity:0' shape=(None, 19) dtype=float32>, <tf.Tensor 'code0_24/Identity:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'code1_24/Identity:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'code2_24/Identity:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'code3_24/Identity:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'code4_24/Identity:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'code5_24/Identity:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'code6_24/Identity:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'code7_24/Identity:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'code8_24/Identity:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'code9_24/Identity:0' shape=(None, 1) dtype=float32>] [<tf.Tensor 'discriminator_input_18:0' shape=(None, 256, 256, 3) dtype=float32>]
Model: "discriminator"
__________________________________________________________________________________________________

Model: "infogan_guppies"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
z_input (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
labels (InputLayer)             [(None, 19)]         0                                            
__________________________________________________________________________________________________
code0 (InputLayer)              [(None, 1)]          0                                            
__________________________________________________________________________________________________
code1 (InputLayer)              [(None, 1)]          0                                            
____________________________________________________________________________________

TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
generator = load_model("infogan_butterflies_darknet_generator.h5")
label = 0
code1 = None
code2 = True
p1 = None
p2 = True
params = (label, code1, code2, p1, p2)
test_generator(generator, params, latent_size=100)